# Wie Programmierung interaktiv und einfach gestaltet wird

## Warum ist Programmieren schwierig?

1. **Black Box-Problem**: Der Code läuft "versteckt" ab, ohne sichtbare Zwischenschritte
2. **Nicht menschenlesbar**: Code ist primär für Maschinen, nicht für Menschen optimiert
3. **Ganzheitliche Ausführung**: Code muss komplett fehlerfrei sein, um überhaupt zu funktionieren
4. **Kontextwechsel**: Ständiges Wechseln zwischen Entwicklungsumgebung und Ausgabe

## Lösungsansatz: Notebooks statt klassischem Quellcode

### Kernprinzipien von Jupyter Notebooks:

- **Modularer Aufbau**: Code wird in einzelne, ausführbare Zellen unterteilt
- **Unabhängige Ausführung**: Jede Zelle kann separat ausgeführt werden
- **Multimodale Inhalte**: Verschiedene Zelltypen (Code, Markdown-Text, Rohdaten)
- **Standardformat**: .ipynb-Dateien (JSON-basiert) speichern Input, Output und Metadaten
- **Professionelle Umgebung**: Project Jupyter bietet die führende Entwicklungsumgebung

# Praktische Demonstration (Python)

## Grundlagen: Zellen und deren Ausführung

Notebooks bestehen aus verschiedenen Zelltypen: **Code**, **Markdown** und **Raw**. 
Zellen werden mit **Shift+Enter** ausgeführt.

In [ ]:
# Einfache Mathematik (implizite Ausgabe)
1 + 1

In [ ]:
# Variablen verwenden (explizite Ausgabe)
a = 1
b = 2
c = a + b
print(c)

In [ ]:
# Mehrere Ergebnisse ausgeben
print('a + b =', c)
print('a * b =', a*b)

## Komplexeres Beispiel: Visualisierung von Klimadaten

### Schritt 1: Bibliotheken importieren

> **Hinweis**: Diese Pakete müssen vorher installiert werden. Nach der Installation den Kernel neu starten!

In [ ]:
# Python-Pakete importieren
# Diese müssen vorab installiert werden (Kernel nach Installation neu starten!)
import numpy as np
import matplotlib.pyplot as plt
import csv

### Schritt 2: Daten herunterladen

Wir laden Klimadaten vom Deutschen Wetterdienst herunter:

In [ ]:
# Klimadaten vom Deutschen Wetterdienst herunterladen
!curl -O https://opendata.dwd.de/climate_environment/CDC/regional_averages_DE/annual/air_temperature_mean/regional_averages_tm_year.txt

### Schritt 3: Variablen definieren

In [ ]:
# Variablen für die Datenverarbeitung definieren
Bundesland = 10  # Spaltenindex für Nordrhein-Westfalen

temp = []  # Liste für Temperaturdaten
year = []  # Liste für Jahreszahlen

### Schritt 4: Daten laden und verarbeiten

In [ ]:
Bundesland = 10  # Spaltenindex für Nordrhein-Westfalen

temp = []
year = []

with open("regional_averages_tm_year.txt", "r") as read_file:
    data = csv.reader(read_file, delimiter=";")
    
    next(data)  # Zeile 1: Überschrift
    header = next(data)  # Zeile 2: Länder

    bundesland_name = header[Bundesland].strip()

    for row in data:
        temp.append(float(row[Bundesland]))
        year.append(float(row[0]))

### Schritt 5: Datenvisualisierung

Erstellen wir eine "Warming Stripes" Darstellung der Temperaturentwicklung:

In [ ]:
# Temperaturstreifen (Warming Stripes)
fig, ax = plt.subplots(figsize=(8, 2))
colors = plt.get_cmap('RdBu_r')((temp - np.min(temp)) / (np.max(temp) - np.min(temp)))
ax.bar(year, 1.0, width=1.0, color=colors)
ax.axis('off')
plt.title(f'Temperaturentwicklung {bundesland_name} (1881–2023)')
plt.tight_layout()

> **Datenquelle**: Deutscher Wetterdienst (DWD)  
> **Inspiriert von**: [Scientists for Future Freiburg](https://www.s4f-freiburg.de/temperaturstreifen/)

## Wie fange ich mit Jupyter an?

### Option 1: Lokale Installation (JupyterLab)

**Kurze Anleitung:**
1. JupyterLab und gewünschte Programmiersprache (Kernel) installieren
2. JupyterLab starten
3. Über Webbrowser verbinden

**Detaillierte Dokumentation:**
- [Offizielle Jupyter-Website](https://jupyter.org/install)
- [JupyterLab Installation](https://jupyterlab.readthedocs.io/en/stable/getting_started/installation.html)
- [JupyterLab starten](https://jupyterlab.readthedocs.io/en/stable/getting_started/starting.html)

### Option 2: Als Service nutzen (JupyterHub)

Viele Universitäten und Forschungseinrichtungen bieten eigene JupyterHub-Instanzen an.

> **Tipp**: Fragen Sie Ihren lokalen IT-Service nach verfügbaren Jupyter-Diensten!

## Zusammenfassung

Jupyter Notebooks revolutionieren das Programmieren durch:
- **Interaktivität**: Sofortiges Feedback bei jeder Code-Ausführung
- **Transparenz**: Sichtbare Zwischenschritte und Ergebnisse
- **Dokumentation**: Code und Erklärungen in einem Dokument
- **Experimentierfreude**: Einfaches Testen und Anpassen von Code-Fragmenten

Dies macht Programmierung zugänglicher, verständlicher und effizienter – besonders für Datenanalyse, Visualisierung und wissenschaftliche Anwendungen.

# Aber was war jetzt mit der Windenergie?

# Windkraft-Vorhersage mit Machine Learning
## Ein Beispiel für Data Literacy im Ingenieurstudium

### Problemstellung: Wie vorhersagbar ist die Windenergie?
Als Ingenieur*in sollen Sie die Energieausbeute eines Windparks vorhersagen können. 
Statt mit vereinfachten Gleichungen zu rechnen, verwenden wir echte Wetterdaten!

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from datetime import datetime, timedelta
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')

# Deutsche Locale für bessere Darstellung
import locale
try:
    locale.setlocale(locale.LC_TIME, 'de_DE.UTF-8')
except:
    pass

## 2. Daten sammeln: Echte Windgeschwindigkeiten aus Deutschland
Wir verwenden die **Bright Sky API** - eine kostenlose Schnittstelle zu Daten des Deutschen Wetterdienstes (DWD).

In [ ]:
# Koordinaten für Nordrhein-Westfalen (Münster)
lat, lon = 51.9606, 7.6261

def get_wind_data(lat, lon, start_date, end_date):
    """
    Lädt Winddaten vom Deutschen Wetterdienst über die Bright Sky API
    """
    url = "https://api.brightsky.dev/weather"
    
    params = {
        'lat': lat,
        'lon': lon,
        'date': start_date.strftime('%Y-%m-%d'),
        'last_date': end_date.strftime('%Y-%m-%d')
    }
    
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        
        # Daten in DataFrame umwandeln
        weather_records = []
        for record in data['weather']:
            weather_records.append({
                'timestamp': pd.to_datetime(record['timestamp']),
                'wind_speed': record.get('wind_speed', 0),  # m/s
                'wind_direction': record.get('wind_direction', 0),  # Grad
                'temperature': record.get('temperature', 0),  # °C
                'pressure': record.get('pressure_msl', 1013),  # hPa
                'humidity': record.get('relative_humidity', 50)  # %
            })
        
        return pd.DataFrame(weather_records)
    else:
        print(f"Fehler beim Abrufen der Daten: {response.status_code}")
        return None

In [ ]:
# Daten für die letzten 30 Tage abrufen
end_date = datetime.now()
start_date = end_date - timedelta(days=30)

print(f"Lade Winddaten für {start_date.strftime('%Y-%m-%d')} bis {end_date.strftime('%Y-%m-%d')}...")
df = get_wind_data(lat, lon, start_date, end_date)

if df is not None:
    print(f"✓ {len(df)} Datensätze erfolgreich geladen!")
    print("\nErstmal schauen, was wir haben:")
    print(df.head())
    
    # Zusätzliche Zeitfeatures erstellen
    df['hour'] = df['timestamp'].dt.hour
    df['day_of_week'] = df['timestamp'].dt.dayofweek
    df['date'] = df['timestamp'].dt.date
else:
    print("❌ Fehler beim Laden der Daten")

## 3. Datenexploration: Was sagen uns die Winddaten?
Bevor wir Modelle bauen, müssen wir verstehen, womit wir arbeiten.

In [ ]:
# Basis-Statistiken
print("📊 Statistiken der Windgeschwindigkeit:")
print(f"Durchschnitt: {df['wind_speed'].mean():.2f} m/s")
print(f"Maximum: {df['wind_speed'].max():.2f} m/s")
print(f"Minimum: {df['wind_speed'].min():.2f} m/s")
print(f"Standardabweichung: {df['wind_speed'].std():.2f} m/s")
print(f"\n📊 Weitere Wetterdaten:")
print(f"Durchschnittliche Temperatur: {df['temperature'].mean():.1f} °C")
print(f"Durchschnittlicher Luftdruck: {df['pressure'].mean():.1f} hPa")
print(f"Durchschnittliche Luftfeuchtigkeit: {df['humidity'].mean():.1f} %")

In [ ]:
# Visualisierung der Windgeschwindigkeiten
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Analyse der Winddaten aus NRW', fontsize=16)

# 1. Zeitreihe der Windgeschwindigkeit
ax1 = axes[0, 0]
ax1.plot(df['timestamp'], df['wind_speed'], alpha=0.7, linewidth=1)
ax1.set_xlabel('Zeit')
ax1.set_ylabel('Windgeschwindigkeit (m/s)')
ax1.set_title('Windgeschwindigkeit über die Zeit')
ax1.grid(True, alpha=0.3)
ax1.tick_params(axis='x', rotation=45)

# 2. Histogramm der Windgeschwindigkeiten
ax2 = axes[0, 1]
ax2.hist(df['wind_speed'], bins=30, alpha=0.7, color='skyblue', edgecolor='black')
ax2.set_xlabel('Windgeschwindigkeit (m/s)')
ax2.set_ylabel('Häufigkeit')
ax2.set_title('Verteilung der Windgeschwindigkeiten')
ax2.axvline(df['wind_speed'].mean(), color='red', linestyle='--', 
            label=f'Durchschnitt: {df["wind_speed"].mean():.1f} m/s')
ax2.legend()

# 3. Tagesverlauf (Durchschnitt pro Stunde)
ax3 = axes[1, 0]
hourly_avg = df.groupby('hour')['wind_speed'].mean()
ax3.plot(hourly_avg.index, hourly_avg.values, marker='o', markersize=6)
ax3.set_xlabel('Stunde des Tages')
ax3.set_ylabel('Durchschn. Windgeschwindigkeit (m/s)')
ax3.set_title('Typischer Tagesverlauf der Windgeschwindigkeit')
ax3.grid(True, alpha=0.3)
ax3.set_xticks(range(0, 24, 3))

# 4. Boxplot nach Wochentag
ax4 = axes[1, 1]
df_boxplot = df.copy()
df_boxplot['weekday'] = df_boxplot['timestamp'].dt.day_name()
weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
df_boxplot['weekday'] = pd.Categorical(df_boxplot['weekday'], categories=weekday_order, ordered=True)
df_boxplot.boxplot(column='wind_speed', by='weekday', ax=ax4)
ax4.set_xlabel('Wochentag')
ax4.set_ylabel('Windgeschwindigkeit (m/s)')
ax4.set_title('Windgeschwindigkeit nach Wochentag')
ax4.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 4. Engineering: Von Windgeschwindigkeit zu Energie
Jetzt wird es ingenieursmäßig! Wir wandeln Windgeschwindigkeit in elektrische Leistung um.

In [ ]:
def wind_to_power(wind_speed, rated_power=2000, cut_in=3, rated_wind=12, cut_out=25):
    """
    Berechnet die Leistung einer Windkraftanlage basierend auf der Windgeschwindigkeit
    
    Parameter:
    - wind_speed: Windgeschwindigkeit in m/s
    - rated_power: Nennleistung der Anlage in kW (default: 2 MW)
    - cut_in: Einschaltgeschwindigkeit in m/s
    - rated_wind: Windgeschwindigkeit bei Nennleistung in m/s
    - cut_out: Abschaltgeschwindigkeit in m/s
    
    Rückgabe:
    - Leistung in kW
    """
    if wind_speed < cut_in:
        # Unterhalb der Einschaltgeschwindigkeit
        return 0
    elif wind_speed >= cut_in and wind_speed < rated_wind:
        # Kubische Beziehung zwischen cut_in und rated_wind
        # Vereinfachte Berechnung mit kubischer Interpolation
        return rated_power * ((wind_speed - cut_in) / (rated_wind - cut_in)) ** 3
    elif wind_speed >= rated_wind and wind_speed < cut_out:
        # Nennleistung
        return rated_power
    else:
        # Oberhalb der Abschaltgeschwindigkeit
        return 0

# Berechnung der theoretischen Energieausbeute
df['power_output'] = df['wind_speed'].apply(wind_to_power)

# Berechnung weiterer Kennzahlen
df['power_coefficient'] = df['power_output'] / 2000  # Kapazitätsfaktor

In [ ]:
print(f"⚡ Theoretische Energieausbeute:")
print(f"Durchschnittliche Leistung: {df['power_output'].mean():.0f} kW")
print(f"Maximale Leistung: {df['power_output'].max():.0f} kW") 
print(f"Minimale Leistung: {df['power_output'].min():.0f} kW")
print(f"Kapazitätsfaktor: {(df['power_output'].mean() / 2000) * 100:.1f}%")

# Energieberechnung
total_hours = len(df)
total_energy = df['power_output'].sum()  # kWh
print(f"\n📊 Energieausbeute über {total_hours} Stunden:")
print(f"Gesamtenergie: {total_energy:,.0f} kWh")
print(f"Das entspricht dem Jahresverbrauch von ca. {total_energy/3500:.0f} Haushalten")

# Volllaststunden
full_load_hours = total_energy / 2000
print(f"Volllaststunden: {full_load_hours:.0f} h")

In [ ]:
# Visualisierung der Leistungskurve
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Leistungskennlinie
wind_range = np.linspace(0, 30, 300)
power_curve = [wind_to_power(w) for w in wind_range]

ax1.plot(wind_range, power_curve, 'b-', linewidth=2, label='Theoretische Kennlinie')
ax1.scatter(df['wind_speed'], df['power_output'], alpha=0.3, s=10, c='red', label='Tatsächliche Daten')
ax1.set_xlabel('Windgeschwindigkeit (m/s)')
ax1.set_ylabel('Leistung (kW)')
ax1.set_title('Leistungskennlinie der Windkraftanlage')
ax1.grid(True, alpha=0.3)
ax1.legend()
ax1.set_xlim(0, 30)
ax1.set_ylim(0, 2200)

# Zeitreihe der Leistung
ax2.plot(df['timestamp'], df['power_output'], alpha=0.7, linewidth=1)
ax2.axhline(y=df['power_output'].mean(), color='red', linestyle='--', 
            label=f'Durchschnitt: {df["power_output"].mean():.0f} kW')
ax2.set_xlabel('Zeit')
ax2.set_ylabel('Leistung (kW)')
ax2.set_title('Leistungsabgabe über die Zeit')
ax2.grid(True, alpha=0.3)
ax2.legend()
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 5. Machine Learning: Können wir die Windenergie vorhersagen?
Hier wird es spannend! Kann ein Algorithmus aus vergangenen Daten die Zukunft vorhersagen?

In [ ]:
# Feature Engineering: Zusätzliche Merkmale erstellen
df['wind_speed_lag1'] = df['wind_speed'].shift(1)  # Windgeschwindigkeit 1h vorher
df['wind_speed_lag6'] = df['wind_speed'].shift(6)  # Windgeschwindigkeit 6h vorher
df['wind_speed_lag12'] = df['wind_speed'].shift(12)  # Windgeschwindigkeit 12h vorher
df['temp_wind_interaction'] = df['temperature'] * df['wind_speed']  # Interaktion
df['pressure_norm'] = (df['pressure'] - df['pressure'].mean()) / df['pressure'].std()  # Normalisiert

# Zyklische Transformation für Stunden (sin/cos encoding)
df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)

# Rolling Statistics
df['wind_speed_rolling_mean_3h'] = df['wind_speed'].rolling(window=3, min_periods=1).mean()
df['wind_speed_rolling_std_3h'] = df['wind_speed'].rolling(window=3, min_periods=1).std()

# Entfernen von NaN-Werten durch Verzögerung
df_clean = df.dropna()

print(f"📊 Feature Engineering abgeschlossen:")
print(f"Ursprüngliche Anzahl Datenpunkte: {len(df)}")
print(f"Nach Bereinigung: {len(df_clean)}")

In [ ]:
# Features und Zielvariable definieren
features = ['wind_speed_lag1', 'wind_speed_lag6', 'wind_speed_lag12', 
            'temperature', 'pressure_norm', 'humidity', 'temp_wind_interaction', 
            'hour_sin', 'hour_cos', 'wind_speed_rolling_mean_3h', 'wind_speed_rolling_std_3h']
X = df_clean[features]
y = df_clean['power_output']

print(f"🤖 Machine Learning Setup:")
print(f"Anzahl Samples: {len(X)}")
print(f"Anzahl Features: {len(features)}")

# Train-Test Split (zeitlich geordnet!)
split_point = int(len(X) * 0.7)
X_train = X[:split_point]
X_test = X[split_point:]
y_train = y[:split_point]
y_test = y[split_point:]

print(f"\nTraining Set: {len(X_train)} Samples")
print(f"Test Set: {len(X_test)} Samples")

# Random Forest Modell trainieren
print("\n🌲 Trainiere Random Forest Modell...")
rf_model = RandomForestRegressor(
    n_estimators=100, 
    max_depth=20,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1
)
rf_model.fit(X_train, y_train)
print("✓ Training abgeschlossen!")

In [ ]:
# Vorhersagen
y_pred = rf_model.predict(X_test)

# Modell-Evaluation
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
mae = np.mean(np.abs(y_test - y_pred))

print(f"📈 Modell-Performance:")
print(f"R² Score: {r2:.3f} (Erklärte Varianz)")
print(f"RMSE: {rmse:.0f} kW")
print(f"MAE: {mae:.0f} kW")
print(f"Relativer Fehler: {(rmse / df['power_output'].mean()) * 100:.1f}%")

# Feature Importance
feature_importance = pd.DataFrame({
    'feature': features,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print(f"\n🔍 Wichtigste Features:")
for idx, row in feature_importance.head().iterrows():
    print(f"  {row['feature']}: {row['importance']:.3f}")

## 6. Ergebnisse visualisieren
Wie gut funktioniert unser Modell? Schauen wir uns die Vorhersagen genauer an.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Machine Learning Ergebnisse: Windenergie-Vorhersage', fontsize=16)

# 1. Vorhersage vs. Tatsächliche Werte
ax1 = axes[0, 0]
ax1.scatter(y_test, y_pred, alpha=0.5, s=20)
ax1.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
ax1.set_xlabel('Tatsächliche Leistung (kW)')
ax1.set_ylabel('Vorhergesagte Leistung (kW)')
ax1.set_title(f'Vorhersage vs. Realität (R² = {r2:.3f})')
ax1.grid(True, alpha=0.3)

# 2. Residuen-Plot
ax2 = axes[0, 1]
residuals = y_test - y_pred
ax2.scatter(y_pred, residuals, alpha=0.5, s=20)
ax2.axhline(y=0, color='r', linestyle='--')
ax2.set_xlabel('Vorhergesagte Leistung (kW)')
ax2.set_ylabel('Residuen (kW)')
ax2.set_title('Residuen-Analyse')
ax2.grid(True, alpha=0.3)

# 3. Feature Importance
ax3 = axes[1, 0]
top_features = feature_importance.head(8)
ax3.barh(range(len(top_features)), top_features['importance'])
ax3.set_yticks(range(len(top_features)))
ax3.set_yticklabels(top_features['feature'])
ax3.set_xlabel('Feature Importance')
ax3.set_title('Wichtigkeit der Eingangsgrößen')
ax3.grid(True, alpha=0.3, axis='x')

# 4. Zeitreihe der Vorhersagen
ax4 = axes[1, 1]
test_timestamps = df_clean['timestamp'].iloc[split_point:]
ax4.plot(test_timestamps, y_test.values, label='Tatsächlich', alpha=0.7)
ax4.plot(test_timestamps, y_pred, label='Vorhersage', alpha=0.7)
ax4.set_xlabel('Zeit')
ax4.set_ylabel('Leistung (kW)')
ax4.set_title('Vorhersage über Zeit')
ax4.legend()
ax4.grid(True, alpha=0.3)
ax4.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 7. Was haben wir gelernt?
Zeit für die wichtigste Frage: Was bedeuten diese Ergebnisse für die Ingenieurspraxis?

In [ ]:
print("🎯 Erkenntnisse aus der Datenanalyse:")
print("=" * 50)

# 1. Windverhältnisse
print("\n1️⃣ WINDVERHÄLTNISSE IN NRW:")
print(f"   • Durchschnittliche Windgeschwindigkeit: {df['wind_speed'].mean():.1f} m/s")
print(f"   • Wind weht {(df['wind_speed'] >= 3).sum() / len(df) * 100:.0f}% der Zeit stark genug für Stromerzeugung")
print(f"   • Optimale Bedingungen (>12 m/s): {(df['wind_speed'] >= 12).sum() / len(df) * 100:.0f}% der Zeit")

# 2. Energieausbeute
print("\n2️⃣ ENERGIEAUSBEUTE:")
print(f"   • Kapazitätsfaktor: {(df['power_output'].mean() / 2000) * 100:.1f}%")
print(f"   • Jährliche Volllaststunden (hochgerechnet): {full_load_hours * 365/30:.0f} h")
print(f"   • Vergleich: Onshore-Wind in Deutschland typisch 1.800-2.500 h/Jahr")

# 3. Vorhersagbarkeit
print("\n3️⃣ VORHERSAGBARKEIT:")
print(f"   • R² Score von {r2:.2f} bedeutet: {r2*100:.0f}% der Varianz erklärt")
print(f"   • Durchschnittlicher Fehler: {mae:.0f} kW ({mae/2000*100:.1f}% der Nennleistung)")
print(f"   • Wichtigster Prädiktor: Wind von vor {1 if 'wind_speed_lag1' in feature_importance.head(1)['feature'].values else 6} Stunde(n)")

# 4. Praktische Implikationen
print("\n4️⃣ PRAKTISCHE IMPLIKATIONEN:")
print("   • Kurzfristvorhersagen (1-6h) sind gut möglich")
print("   • Windgeschwindigkeit der letzten Stunden ist bester Prädiktor")
print("   • Wetterparameter (Temp, Druck) verbessern Vorhersage nur marginal")
print("   • Netzstabilität: Schwankungen müssen ausgeglichen werden")

# 5. Data Science für Ingenieure
print("\n5️⃣ DATA SCIENCE FÜR INGENIEURE:")
print("   • Reale Daten zeigen komplexeres Verhalten als Lehrbuchmodelle")
print("   • Machine Learning kann Muster erkennen, die wir übersehen")
print("   • Aber: Physikalisches Verständnis bleibt essentiell!")
print("   • Datenqualität ist entscheidend für gute Vorhersagen")

print("\n" + "=" * 50)
print("💡 FAZIT: Data Literacy macht Sie zu besseren Ingenieuren!")

In [ ]:
# Bonus: Wirtschaftliche Betrachtung
strompreis = 0.08  # €/kWh (Börsenpreis)
print("\n💰 WIRTSCHAFTLICHE BETRACHTUNG:")
print(f"Ertrag in {total_hours/24:.0f} Tagen: {total_energy * strompreis:,.0f} €")
print(f"Hochgerechnet auf ein Jahr: {total_energy * strompreis * 365/30:,.0f} €")
print(f"Bei Investitionskosten von 3 Mio. € amortisiert sich die Anlage nach ca. {3000000 / (total_energy * strompreis * 365/30):.1f} Jahren")